# Visualisation of LFP & Calcium traces if no subsessions

### Load LFP and packages

In [ ]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
import json
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
import sys 
import pickle

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

%matplotlib widget

In [ ]:
cd "C:/Users/Manip2/SCRIPTS/Code python audrey/code python aurelie/interfaceJupyter/minian"

In [ ]:
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

In [ ]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

### for both sessions and subsessions (to work on it)

In [ ]:
"""
folder_base = Path(dpath) 

nb_sessions = sum(1 for p in folder_base.iterdir() if p.is_dir() and p.name.startswith("session"))
try:
    mfile = open(folder_base.parent / f'mappingsAB.pkl', 'rb')
    mapping = pickle.load(mfile)
except:
    mfile = open(folder_base.parent / f'mappings.pkl', 'rb')
    mapping = pickle.load(mfile)

sessions = []
subsessions = []
nb_minian_total=0
dict_Calcium = {}
dict_Spike = {}
dict_Scoring = {}
dict_Stamps = {}
dict_TodropFile = {}


session= 'session' + str(y)
print(session)
sessions.append(session)


folder_mini = folder_base / f'V4_Miniscope'
nb_subsessions = sum(1 for p in folder_mini.iterdir() if p.is_dir() and p.name.startswith("session"))
StampsFile = folder_base / f'SynchroFile.xlsx'

ScoringFile = folder_base / f'OpenEphys/ScoredSleep.npy'
SleepScoredTS=np.load(ScoringFile)

filename2 = folder_base / f'OpenEphys/RawDataChannelExtractedDS.npy'
All = np.load(filename2, mmap_mode= 'r')

Channels = '//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/LFPChannels_perMice.xlsx' 
allchannels = pd.read_excel(Channels)
mice = os.path.basename((os.path.dirname(folder_base)))

PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])
EMGch1=int(allchannels[mice][3])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
S1  =  All[:, S1ch1]-All[:, S1ch2] 
EMG  =  All[:, EMGch1]

if nb_subsessions!=0:
    for x in range(1, nb_subsessions+1):            
        subsession= "session"  + str(y) + str(x)
        subsessions.append(subsession)    
        minian_ds = open_minian(folder_mini / subsession / f'minian')      # OR minianAB
        dict_Calcium[subsession] = minian_ds['C'] # calcium traces 
        dict_Spike[subsession] = minian_ds['S'] # estimated spikes
        dict_Scoring[subsession]  = np.load(ScoringFile)
        dict_Stamps[subsession]  = pd.read_excel(StampsFile)
        try:
            TodropFile = folder_mini / subsession / f'minian/TodropFileAB.json'
            with open(TodropFile, 'r') as f:
                unit_to_drop = json.load(f)
                dict_TodropFile[subsession]  = unit_to_drop
        except:
            TodropFile = folder_mini / subsession / f'minian/TodropFile.json'
            with open(TodropFile, 'r') as f:
                unit_to_drop = json.load(f)
                dict_TodropFile[subsession]  = unit_to_drop

else:
    folder_mini = folder_base / f'V4_Miniscope'
    minian_ds = open_minian(folder_mini / f'minian')
    Calcium = minian_ds['C'] # calcium traces 
    Spike = minian_ds['S'] # estimated spikes

    Stamps= pd.read_excel(StampsFile)
    StartTime = list(Stamps[0])[0]
    minian_freq=list(Stamps[0])[2]

    try:
        TodropFile = folder_mini / f'minian/TodropFileAB.json'
        with open(TodropFile, 'r') as f:
            unit_to_drop = json.load(f)
            dict_TodropFile = unit_to_drop
    except:
        TodropFile = folder_mini / f'minian/TodropFile.json'
        with open(TodropFile, 'r') as f:
            unit_to_drop = json.load(f)
            dict_TodropFile  = unit_to_drop
"""


### if no subsessions

In [ ]:
folder_base = Path(dpath) 

filename2 = folder_base / f'RawDataChannelExtractedDS.npy'
All = np.load(filename2, mmap_mode= 'r')

ScoringFile = folder_base / f'ScoredSleep.npy'
SleepScoredTS=np.load(ScoringFile)

Channels = '//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/LFPChannels_perMice.xlsx' 

folder_mini = folder_base.parent / f'V4_Miniscope'
minian_ds = open_minian(folder_mini / f'minian')
Calcium = minian_ds['C'] # calcium traces 
Spike = minian_ds['S'] # estimated spikes

StampsFile = folder_base.parent / f'SynchroFile.xlsx'
Stamps= pd.read_excel(StampsFile)
StartTime = list(Stamps[0])[0]
minian_freq=list(Stamps[0])[2]

try:
    TodropFile = folder_mini / f'minian/TodropFileAB.json'
    with open(TodropFile, 'r') as f:
        unit_to_drop = json.load(f)
except:
    TodropFile = folder_mini / f'minian/TodropFile.json'
    with open(TodropFile, 'r') as f:
        unit_to_drop = json.load(f)

freqLFP=1000

def readfile(file):
  with open(file, encoding="utf-8") as file:
    list = []
    for row in file:
      temp = row.split()
      list.append(temp[:])
    return list

mice = os.path.basename(os.path.dirname(os.path.dirname(folder_base)))
allchannels = pd.read_excel(Channels)

PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])
EMGch1=int(allchannels[mice][3])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
S1  =  All[:, S1ch1]-All[:, S1ch2] 
EMG  =  All[:, EMGch1]

# Upscale Scoring
indices = np.linspace(0, len(SleepScoredTS) - 1, len(SleepScoredTS))
new_indices = np.linspace(0, len(SleepScoredTS) - 1, len(EMG))
interpolated_func = interp1d(indices, SleepScoredTS, kind='linear')
SleepScoredTS_upscaled = interpolated_func(new_indices)
SleepScoredTS_upscaled = np.round(SleepScoredTS_upscaled * 2) / 2

D = Calcium['unit_id']
copyD = list(D.copy())
for r in range(len(unit_to_drop)):
    elem = unit_to_drop[r]
    copyD.remove(elem)
unit_to_keep = copyD
Calcium = np.array(Calcium.loc[unit_to_keep,:])
Spike = np.array(Spike.loc[unit_to_keep,:])

# Delay and Upscale CalciumTraces
nan_matrix=np.full((len(Calcium), int(StartTime*minian_freq)), np.nan)
Calcium_aligned=np.concatenate((nan_matrix.T, Calcium.T), axis=0)
Spike_aligned=np.concatenate((nan_matrix.T, Spike.T), axis=0)

scalefactor=freqLFP/minian_freq
upscaled_Calcium = np.repeat(Calcium_aligned, scalefactor, axis=0)
upscaled_Spike = np.repeat(Spike_aligned, scalefactor, axis=0)

In [ ]:
OldSWSdetection=folder_base / f'SWRproperties_AB.csv'
SWR_prop = pd.read_csv(OldSWSdetection, index_col=0)            
SWR_prop=SWR_prop.values
SWR_peak = np.transpose(SWR_prop[:,0].astype(int))
SWR_start = np.transpose(SWR_prop[:,3].astype(int))
SWR_end = np.transpose(SWR_prop[:,4].astype(int))

OldS1spdldetection=folder_base / f'Spindlesproperties_S1_AB.csv'
S1spdl_prop = pd.read_csv(OldS1spdldetection, index_col=0)            
S1spdl_prop=S1spdl_prop.values
S1spdl_peak = np.transpose(S1spdl_prop[:,0].astype(int))
S1spdl_start = np.transpose(S1spdl_prop[:,3].astype(int))
S1spdl_end = np.transpose(S1spdl_prop[:,4].astype(int))

OldPFCspdldetection=folder_base / f'Spindlesproperties_PFC_AB.csv'
PFCspdl_prop = pd.read_csv(OldPFCspdldetection, index_col=0)            
PFCspdl_prop=PFCspdl_prop.values
PFCspdl_peak = np.transpose(PFCspdl_prop[:,0].astype(int))
PFCspdl_start = np.transpose(PFCspdl_prop[:,3].astype(int))
PFCspdl_end = np.transpose(PFCspdl_prop[:,4].astype(int))

# EphyViewer
### LFPs, Calcium traces, Spikes

In [17]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, CsvEpochSource, EpochEncoder,EpochViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import InMemorySpikeSource


app = mkQApp()
win = MainViewer(debug=True, show_auto_scale=True)

# LFPs

sample_rate = freqLFP
t_start = 0.
scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: SWR_start, 5: SWR_end}
scatter_channels = {0: [1], 1: [1], 2: [0], 3: [0], 4: [2], 5: [2]}
combined = np.stack([(S1), (PFC), (CA1)/5, (EMG)/5], axis = 1)
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#FFFFFF', 5: '#222222'}, channel_names=['S1','PFC', 'CA1', 'EMG', 'Scoring'])

view1 = TraceViewer(source=source)

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#88FF88' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#8888FF'
view1.by_channel_params['ch2', 'color'] = '#FF8888'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'

# Calcium traces 

view2 = TraceViewer.from_numpy(zscore(Calcium.T), float(minian_freq), round(StartTime,2), 'Calcium Traces')
view2.params['scale_mode'] = 'same_for_all'
view2.auto_scale()


# Spike traces 

all_spikes = []
for c,row in enumerate(upscaled_Spike.T):
    peaks, _ = find_peaks(row, height=np.nanstd(row))
    all_spikes.append({ 'time':peaks/freqLFP, 'name':'Unit#{}'.format(c) })
source = InMemorySpikeSource(all_spikes=all_spikes)
view3 = ephyviewer.SpikeTrainViewer(source=source, name='spikes')

#add them to mainwindow

win.add_view(view1)
win.add_view(view2)
win.add_view(view3)

#Run

win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.013623952865600586 s
refresh duration for  0.01566290855407715 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh duration for  0.0 s
refresh duration for Calcium Traces 0.0 s
refresh duration for spikes 0.0 s
refresh dura

0

### LFPs, Spikes, FFT

In [ ]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, CsvEpochSource, EpochEncoder,EpochViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import InMemorySpikeSource

app = mkQApp()
win = MainViewer(debug=True, show_auto_scale=True)

# LFPs

sample_rate = freqLFP
t_start = 0.
scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: SWR_start, 5: SWR_end}
scatter_channels = {0: [1], 1: [1], 2: [0], 3: [0], 4: [2], 5: [2]}
combined = np.stack([(S1), (PFC), (CA1)/5, (EMG)/5], axis = 1)
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#FFFFFF', 5: '#222222'}, channel_names=['S1','PFC', 'CA1', 'EMG', 'Scoring'])

view1 = TraceViewer(source=source)

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#88FF88' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#8888FF'
view1.by_channel_params['ch2', 'color'] = '#FF8888'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 60
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz

view3.by_channel_params['ch0', 'clim'] = 300
view3.by_channel_params['ch1', 'clim'] = 300
view3.by_channel_params['ch2', 'clim'] = 300
view3.by_channel_params['ch0', 'visible'] = True
view3.by_channel_params['ch1', 'visible'] = False
view3.by_channel_params['ch2', 'visible'] = False
view3.by_channel_params['ch3', 'visible'] = False


# Spike traces 

all_spikes = []
for c,row in enumerate(upscaled_Spike.T):
    peaks, _ = find_peaks(row, height=np.nanstd(row))
    all_spikes.append({ 'time':peaks/freqLFP, 'name':'Unit#{}'.format(c) })
source = InMemorySpikeSource(all_spikes=all_spikes)
view2 = ephyviewer.SpikeTrainViewer(source=source, name='spikes')


# Add them to mainwindow

win.add_view(view1)
win.add_view(view2)
win.add_view(view3)

#Run

win.show()
app.exec_()